I recently went through the Deeplearning.ai hugging face course and really liked reading through the sentence embedding module. In finding similarities amoung sentences, I wondered how similar the different models (OpenAI, Mistral, and Anthropic) are. This notebook is an exploration given the same inputs, how similar are the responses from each model.

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

In [ ]:
 !pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━


# Install and set up LLM providers

In [ ]:
# install from PyPI
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


Set up OpenAI API call

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY'),
)

completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message.content)

In the realm of code where algorithms reign,
There's a concept that drives many programmers insane.
Recursion is its name, a loop within a loop,
A mystic dance where the code does swoop.

Like a Russian doll that hides within,
A function calls itself to begin.
A never-ending journey through the rabbit hole,
Repeating tasks with a mystical goal.

With each recursive call, the solution unfolds,
Breaking down problems in recursive codes.
A loop that spirals deeper into the abyss,
Tackling complexities with recursive bliss.

But be wary, dear coder, in this recursive land,
For infinite loops are always close at hand.
Base cases hold the key to break the chain,
And end the recursive dance, without a stain.

So embrace the beauty of recursion's art,
And let your code unravel from the heart.
For in this tangled web of loops untold,
Recursion's magic will forever hold.


Set up Anthropic API call

In [ ]:
import anthropic
from google.colab import userdata

anthropic_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=userdata.get('ANTHROPIC_API_KEY'),
)
message = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude. Tell me a short story!"}
    ]
)

In [ ]:
print(message.content[0].text)

Once upon a time, in a quaint little village nestled in the heart of a lush green forest, there lived a young girl named Lily. Lily had a special gift: she could communicate with animals. Every day, she would venture into the woods and spend hours talking to her furry and feathered friends.

One particularly sunny morning, Lily met a distressed squirrel who told her that the forest was in danger. An evil sorcerer had cast a spell, causing the trees to wither and the rivers to dry up. Determined to save her beloved forest, Lily embarked on a quest to find the sorcerer and break the spell.

As she journeyed deeper into the woods, Lily encountered many challenges. She crossed treacherous ravines, outsmarted cunning predators, and navigated through enchanted mists. Along the way, she made new friends – a wise old owl, a brave deer, and a mischievous fox – who joined her on her mission.

Finally, they reached the sorcerer's castle. Using her wit and the help of her animal companions, Lily c

Set up Mistral API Call

In [ ]:
from google.colab import userdata

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-tiny"

client = MistralClient(userdata.get('MISTRAL_API_KEY'))

mistral_chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="What is the best French cheese?")],
)
print(mistral_chat_response.choices[0].message.content)

Determining the "best" French cheese can be subjective as it depends on personal preferences. Here are some popular and highly regarded French cheeses:

1. Roquefort: A blue-veined cheese made from sheep's milk, known for its distinctive pungent smell and creamy texture.

2. Comté: A nutty and rich tasting cheese made from unpasteurized cow's milk, it comes from the Franche-Comté region in eastern France.

3. Camembert: A soft and creamy cheese with a white rind and a pungent, earthy flavor, made from cow's milk in the Normandy region.

4. Brie de Meaux: A soft and creamy cheese with a bloomy rind and a mild, buttery flavor, originating from the Marne region in northeastern France.

5. Munster: A pungent, smelly, and runny cheese made from cow's milk with a wash of marc (brandy), originating from the Alsace region.

6. Chaource: A soft, creamy, and mild-tasting cheese with a bloomy rind, made from cow's milk in the region of the same name.

7. Reblochon: A semi-soft, rich, and buttery 

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate Text from providers

Generate responses from the 3 different services based on the same prompt.

In [ ]:
prompt = 'I am a developer and I want to learn how to code. My speciality is in backend development and I want to get better at front end. Where do I start?'

In [ ]:
mistral_chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)],
)

In [ ]:
anthropic_message = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [ ]:
openai_completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt}
  ]
)

Get the vector calculations of the messages.

In [ ]:
embeddings_mistral = model.encode(mistral_chat_response.choices[0].message.content, convert_to_tensor=True)

In [ ]:
embeddings_anthropic = model.encode(anthropic_message.content[0].text, convert_to_tensor=True)

In [ ]:
embeddings_openai = model.encode(openai_completion.choices[0].message.content, convert_to_tensor=True)

Calculate cosine similarity among the three different providers

In [ ]:
from sentence_transformers import util

In [ ]:
cosine_scores_mistral_anthropic = util.cos_sim(embeddings_mistral, embeddings_anthropic)

In [ ]:
print(cosine_scores_mistral_anthropic)

tensor([[0.9224]])


In [ ]:
cosine_scores_mistral_openai = util.cos_sim(embeddings_mistral, embeddings_openai)
print(cosine_scores_mistral_openai)

tensor([[0.8241]])


In [ ]:
cosine_scores_openai_anthropic = util.cos_sim(embeddings_openai, embeddings_anthropic)

In [ ]:
print(cosine_scores_openai_anthropic)

tensor([[0.8696]])
